In [1]:
import os 
import sys 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from IPython.display import clear_output
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../.env')

print(os.environ['SUBGRAPH_URL'])

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    stack_order_expr, 
    wide_to_longwide, 
    chart_stack_area_overlay_line_timeseries,
)
from utils_notebook.queries import adjust_precision, QueryManager
from utils_notebook.testing import validate_season_series
from utils_notebook.css import css_tooltip_timeseries_multi_colored
from utils_notebook.queries import QueryManager

https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk


In [2]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

In [15]:
from utils_notebook.utils import add_silo_token_name_adjust_precision

farmer_address = "" # Input farmer address here 
silo_deposits = bs.Query.siloDeposits(
    first=100, orderBy="season", orderDirection="asc", where={
        "farmer_": {"id": farmer_address}
    }
)
df = sg.query_df([
    silo_deposits.season, 
    silo_deposits.updatedAt, 
    silo_deposits.bdv, 
    silo_deposits.removedBDV, 
    silo_deposits.removedTokenAmount, 
    silo_deposits.token, 
    silo_deposits.tokenAmount,
])
df = remove_prefix(df, "siloDeposits_")
df.updatedAt = pd.to_datetime(df.updatedAt, unit="s")
add_silo_token_name_adjust_precision(df, "token", ['tokenAmount', 'removedTokenAmount'])
df.removedBDV /= 1e6
df.bdv /= 1e6 
df['bdv_remaining'] = df.bdv - df.removedBDV
df['amount_remaining'] = df.tokenAmount - df.removedTokenAmount

In [2]:
# df.head(50)

In [20]:
from utils_notebook.utils import add_silo_token_name_adjust_precision

add_deposits = bs.Query.addDeposits(
    first=100, orderBy="season", orderDirection="asc", where={
        "account": farmer_address
    }
)
df = sg.query_df([
    add_deposits.season,
    add_deposits.timestamp,
    add_deposits.token,
    add_deposits.amount,
    add_deposits.bdv,
])
df = remove_prefix(df, "addDeposits_")
df.timestamp = pd.to_datetime(df.timestamp, unit="s")
add_silo_token_name_adjust_precision(df, "token", ['amount'])
df.bdv /= 1e6 

In [1]:
# df.head(50)